# LBP + k-NN Classifier for Aerial Scene Classification

This notebook extracts **Local Binary Pattern (LBP)** features from aerial images and uses **k-Nearest Neighbors (k-NN)** for classification.

In [1]:

import cv2
import numpy as np
import os
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from tqdm import tqdm


In [2]:

# Parameters
dataset_path = '../Aerial_Landscapes'
RADIUS = 1
N_POINTS = 8 * RADIUS
METHOD = 'uniform'
LBP_HIST_SIZE = N_POINTS + 2


In [3]:
def load_dataset_split(split_dir):
    images, labels = [], []
    class_names = sorted(os.listdir(split_dir))
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(split_dir, class_name)
        for img_path in glob(os.path.join(class_path, '*.jpg')):
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
                labels.append(label)
    return images, labels, class_names

# Load from pre-split folders
train_images, train_labels, class_names = load_dataset_split('../split_data/train')
val_images, val_labels, _ = load_dataset_split('../split_data/val')
test_images, test_labels, _ = load_dataset_split('../split_data/test')



In [4]:

def extract_lbp_features(images):
    lbp_features = []
    for img in tqdm(images):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, N_POINTS, RADIUS, METHOD)
        (hist, _) = np.histogram(lbp.ravel(),
                                 bins=np.arange(0, LBP_HIST_SIZE + 1),
                                 range=(0, LBP_HIST_SIZE))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_features.append(hist)
    return np.array(lbp_features)

X = extract_lbp_features(train_images)
y = np.array(train_labels)


100%|██████████| 8400/8400 [02:08<00:00, 65.57it/s]


In [5]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:

# Train k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(metric='euclidean')

In [11]:

# Evaluate the model
y_pred = knn.predict(X_test_scaled)

print("——— 评估结果 ———")
print("准确率 (Accuracy):", accuracy_score(y_test, y_pred))
print("精确率 (Precision):", precision_score(y_test, y_pred, average='macro'))
print("召回率 (Recall):", recall_score(y_test, y_pred, average='macro'))
print("F1分数 (F1-Score):", f1_score(y_test, y_pred, average='macro'))
print("\n混淆矩阵:")
print(confusion_matrix(y_test, y_pred))
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))


——— 评估结果 ———
准确率 (Accuracy): 0.5041666666666667
精确率 (Precision): 0.508956710036231
召回率 (Recall): 0.5009182569199897
F1分数 (F1-Score): 0.4991809763188667

混淆矩阵:
[[81  4 12  2  6  0  2  5  1  7  0  1  2  1  2]
 [12 38  2 12  0  0  0  8  0 14  5  2 12  1  3]
 [12  3 57  3 14  0  2  0 11  1  1  5  0  8  7]
 [ 1 13  0 47  1  0  0  3  0  3  7  1 18  5  0]
 [ 7  4 11  0 63  3  3  0  1 10  0  0  0  2  4]
 [ 0  0  1  0  5 95  9  0  2  1  0  0  0  0  0]
 [ 3  2  2  0 14 12 66  2  1  1  0  0  0  5  0]
 [ 4 22  4 11  0  1  1 39  1  3  4  1  9  8  3]
 [ 5  5 12  0  3  2  0  0 52  5  0 17  0  3 15]
 [ 4  5  1  4 10  5  1  0  1 60  0  0  1  4 10]
 [ 9  7  4 20  0  1  0  6  2  0 47  1  3 12  6]
 [ 3  0  8  2  0  0  0  0 20  0  5 76  0  4  0]
 [ 2 11  0 21  0  0  0 16  0  4  1  0 29  8  2]
 [ 2  8  1  8  1  2  0  5  1 11  4  0  4 68  1]
 [ 8  9 12  4  2  1  3  2 13 15  0  3  0 10 29]]

分类报告:
              precision    recall  f1-score   support

 Agriculture     0.5294    0.6429    0.5806       126
    